# Creating a classification map on Google earth engine
In this notebook we will use Google Earth Engine to perform a classification. Google earth engine is a great resource for accessing and processing data. However, there are various quotas that limit the amount of processing and RAM you can use for free. To address some of thees limitation we will be using random subsets of our data and reduce the complexity of our models. 

In [ ]:
#import packages
import geopandas as gpd, pandas as pd, os, numpy as np
import ee, geemap
from geemap import ml

### Authenticate into Earth Engine

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-jshogland') #you will want to select your personal cloud project

## Create definitions for the median and medoid procedures

In [ ]:
def maskL8sr(image):
    # Bit 0 - Fill
    # Bit 1 - Dilated Cloud
    # Bit 2 - Cirrus
    # Bit 3 - Cloud
    # Bit 4 - Cloud Shadow
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)
    # Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    #Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, overwrite=True).addBands(thermalBands, overwrite=True).updateMask(qaMask).updateMask(saturationMask)

def median_mosaic(image,fltr=None,refl_bands=['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']):
    if(fltr is None):
        inCollection = image.filter(fltr).select(refl_bands)
    else:
        inCollection = image.filter(fltr).select(refl_bands)

    return inCollection.median()

def _medoid(col):
    median = ee.ImageCollection(col).median()
    diff=ee.Image(col).subtract(median).pow(ee.Image.constant(2))
    return diff.reduce('sum').addBands(col)


def medoid_mosaic(image, fltr,refl_bands=['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']):
    if(fltr is None):
        inCollection = image.filter(fltr).select(refl_bands)
    else:
        inCollection = image.filter(fltr).select(refl_bands)

    medoid = inCollection.map(_medoid)
    medoid = ee.ImageCollection(medoid).reduce(ee.Reducer.min(7)).select([1,2,3,4,5,6], refl_bands)
    return medoid



## Set various variable and create the medoid surface on ee

In [ ]:
#make lists fo band names for selections
lc8_bands = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10', 'QA_PIXEL']#landsat band names
tgt_bands = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'TEMP', 'QA_PIXEL']#common band names
refl_bands = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']#bands we care about

#specify start and end dates for the image filter
startDate = '2021-01-01'
endDate = '2024-07-01'

#Specify julian dates for filter. Here we want to select sunny months
julianStart1 = 350# Starting Julian Date (for landsat median cloud free )
julianEnd1 = 365
julianStart2 = 1
julianEnd2 = 150# Ending Julian date (for landsat median cloud free)

#define the study area extent from our convex hull
#geo=geemap.gdf_to_ee(gpd.GeoDataFrame(geometry=chul)) #convert our convex hull into a ee feature class object

#make the ee collection
l8_col=ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')

#set various filters
#f_bnds=ee.Filter.bounds(geometry=geo)
f_date=ee.Filter.date(startDate,endDate)
f_cr1=ee.Filter.calendarRange(julianStart1,julianEnd1)
f_cr2=ee.Filter.calendarRange(julianStart2,julianEnd2)
f_or=ee.Filter.Or(f_cr1,f_cr2)
f_and=ee.Filter.And(f_date,f_or)

#use our filter on the landsat collection
l8=l8_col.filter(f_and).map(maskL8sr)
l8r=l8.select(lc8_bands,tgt_bands)

#call the medoid function
medoid = medoid_mosaic(l8r,fltr=f_and,refl_bands=refl_bands)

#get the elevation data
dem = ee.Image("USGS/SRTMGL1_003")

#Create Raster Predictors
pred_rs=ee.Image([medoid,dem])

## Make Random Forest Model
#### Make the sklearn model and convert it into a google earth engine model
We will be using our cleaned data. If you have not already worked through the [Summarizing plot data](./Summarizing_plot_data.ipynb) notebook, please do so before continuing. Our response and predictor variables are as follows:-
- response = Use2
- predictors = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2','elevation']

In [ ]:
from sklearn.ensemble import RandomForestClassifier

df=pd.read_csv('./plot_subplot_data.csv')
resp='Use2'
pred=['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2','elevation']

#mapping to integers
uvls=df['Use'].unique()
cdic=dict(zip(uvls,np.arange(uvls.shape[0])))
df['Use2']=df['Use'].map(cdic)

#subset the data to send the model to Google EE
ss=int(df.shape[0]*0.1)
sdf=df.sample(ss)
X=sdf[pred]
y=sdf[resp]

rf=RandomForestClassifier(n_estimators=10,max_samples=0.75,random_state=0,oob_score=True) # reduce the number of models for ee
rf.fit(X,y)
print('Label Dictionary:')
display(cdic)
print('OOB = ',rf.oob_score_)

### Convert the sklearn model to strings to upload into a classifier using Geemap's ml module

In [ ]:
rf_str=ml.rf_to_strings(rf,pred)

### Convert the strings into a ee classifier using Geemap's ml module

In [ ]:
clsf=ml.strings_to_classifier(rf_str)

### Create the ee classified imgage using Earth Engine functions

In [ ]:
classified = pred_rs.select(pred).classify(clsf)

## Visualize the Medoid image and classification

In [ ]:
Map = geemap.Map()

Map.set_center(lon=-83.7534,lat=9.7489,zoom=8)

Map.addLayer(
    pred_rs,
    {"bands": ["RED", "GREEN", "BLUE"], "min": 0, "max": 0.25, "gamma": 1.5},
    "Medoid",
)
Map.addLayer(
    classified,
    {"min": 0, "max": 7, "palette": ["green", "tan", "blue","white","yellow","lightgreen","grey","grey"]},
    "classification",
)

Map

## Exercise 1: Processing
- Where did we create the random forest model? (google server or locally)
- Where did we create the imagery? (google server or locally)
- Where did we extract the point data? (google server or locally)
- Why did we subset our data?
- How did we transfer our random forest model to google earth engine?
- How are we processing the imagery on Google Earth Engine yet we are able to see it on our local machine (we are not using a web browser)?
- How could we save this map as an HTML file?
- How accurate is our map?
